# Problemas

Nesta prática iremos usar tudo que aprendemos durante o módulo.
Logo, **seu objetivo é determinar e implementar um modelo para cada problema.**

Lembre-se de definir:

1. o Dataloader, tratando a forma de ler as imagens de cada dataset, experimentando transformações diferentes (resize, crop, flips e etc.)
1. uma arquitetura (tentem usar tanto arquiteturas existentes como propor novas usando camadas de convolução, pooling, e densas),
1. uma função de custo
1. um algoritmo de otimização (agora, como os problemas são maiores, será possível notar mais claramente a diferença entre diferentes algoritmos).





In [4]:
# %load_ext nbproxy
# %pip install torchsummary

In [5]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import PIL
from PIL import Image
from torch import optim
from torchsummary import summary

# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))

In [6]:
# funções básicas

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    net.eval()
    with torch.no_grad():
      for X, y in data_iter:
          #y = y.astype('float32')
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)

# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        net.train()
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))


## Problema 1 - Exemplo

Neste problema, classificaremos imagens histólogica do dataset [*Colorectal Histology*](https://www.kaggle.com/kmader/colorectal-histology-mnist).
Neste caso, vamos receber imagens com tamanho de $150\times 150$ pixels e classificá-las entre 8 classes:

1. tumor
1. stroma
1. complex
1. lympho
1. debris
1. mucosa
1. adipose
1. empty

In [7]:
!wget https://www.dropbox.com/s/k0f6vxyhcr6gh1r/Kather_texture_2016_image_tiles_5000.zip
!unzip -q Kather_texture_2016_image_tiles_5000.zip

--2025-07-01 19:58:29--  https://www.dropbox.com/s/k0f6vxyhcr6gh1r/Kather_texture_2016_image_tiles_5000.zip
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:601d:18::a27d:512, 162.125.5.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:601d:18::a27d:512|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/eyds9w2eqfkq7rx86z4oj/Kather_texture_2016_image_tiles_5000.zip?rlkey=w9zrj0hx7dflh7913i4ds0fi7 [following]
--2025-07-01 19:58:30--  https://www.dropbox.com/scl/fi/eyds9w2eqfkq7rx86z4oj/Kather_texture_2016_image_tiles_5000.zip?rlkey=w9zrj0hx7dflh7913i4ds0fi7
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbc877f81b3abe7066d4f47493e.dl.dropboxusercontent.com/cd/0/inline/CssdYp4DeipJfq4ar3A-VDTVakyBqCoZQAaRkaYVxEGIXaI-p6URZ9rPTiRkx8VwdQT3gSgZ29GnfxIeJdxEAzpyg97_BQYbRz9EbCYuHlOOsdk4vVIO_h22-4WqH1dZgCI/file# [following]
--2025-07-01 19:58:30-

In [8]:
class HistologyDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform, train=False, calc_norm=True, has_norm=True):
        self.root = root
        self.train = train
        self.calc_norm = calc_norm
        self.has_norm = has_norm
        self.le = {'tumor': 0, 'stroma': 1, 'complex': 2, 'lympho': 3,
                   'debris': 4, 'mucosa': 5, 'adipose': 6, 'empty': 7} # dicionário para definir o label de cada classe
        self.transform = transform
        self.load_images()

    def load_images(self):
        self.img_list, self.labels = self.read_images(root=self.root)

    def read_images(self, root):
        # Leitura das imagens do dataset
        # para este caso, o dataset divide em pastas as imagens de cada classe correspondente
        # portanto, vamos percorrer essas pastas, adicionando as primeiras 500 imagens para o conjunto de treino
        # o restante das imagens (a partir da 500) são adicionadas na validação
        # o label é definido de acordo com o nome da pasta pelo dicionário self.le definido acima
        # por exemplo: a pasta 01_TUMOR vai ser correspondente ao self.le['tumor'], que é igual a 0
        img_list, labels = [], []
        if self.train is True:
          for folder in os.listdir(self.root):
            for num, img_name in enumerate(os.listdir(os.path.join(self.root, folder))):
                if num < 500:
                  img_list.append(os.path.join(self.root, folder, img_name))
                  labels.append(self.le[folder.split('_')[1].lower()])
        else:
          for folder in os.listdir(os.path.join(self.root)):
            for num, img_name in enumerate(os.listdir(os.path.join(self.root, folder))):
                if num >= 500:
                  img_list.append(os.path.join(self.root, folder, img_name))
                  labels.append(self.le[folder.split('_')[1].lower()])

        return img_list, labels

    def __getitem__(self, item):
        # retorna uma imagem para o treino/teste
        if self.has_norm is True:
            # normaliza a imagem se has_norm for setado como True
            cur_img = self.normalize_image(self.transform(Image.open(self.img_list[item])))
        else:
            # apenas converte a imagem para tensor, sem normalizar
            cur_img = self.transform(Image.open(self.img_list[item]))
        cur_label = self.labels[item]
        return cur_img, cur_label

    def __len__(self):
        return len(self.img_list)

    def normalize_image(self, img):
        # normaliza uma imagem
        # se calc_norm for True, normaliza pela subtração da média dividida pelo desvio para cada canal da imagem
        # se calc_norm for False, normaliza pelos valores pré-definidos de média e desvio padrão
        if self.calc_norm is True:
            for i in range(img.shape[0]):
                mu = img[i, :, :].mean()
                std = img[i, :, :].std()
                img[i, :, :] = ((img[i, :, :] - mu) / std)
        else:
            img = torchvision.transforms.functional.normalize(img,
                                                mean=torch.Tensor([0.485, 0.456, 0.406]),
                                                std=torch.Tensor([0.229, 0.224, 0.225]))
        return img


def load_data(dataset, root, batch_size, resize=None):
    # o transformer define a sequência de transformações que serão aplicadas na imagem
    # neste caso, a sequência é um redimensionamento da imagem (caso a variável resize seja definida)
    # seguido de uma transformação para tensor
    # várias outras transformações estão disponíveis no Pytorch, como crops, flips, espelhamento e etc.
    transformer = []
    if resize is not None:
        transformer += [torchvision.transforms.Resize(size=(resize,resize))]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    train = dataset(root=root, transform=transformer, train=True) #obtem dataset de treino
    test = dataset(root=root, transform=transformer, train=False) #obtem dataset de validação
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = torch.utils.data.DataLoader(train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers) # criação do dataloader de treino
    test_iter = torch.utils.data.DataLoader(test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers) # criação do dataloader de teste
    return train_iter, test_iter

# carregamento do dado
batch_size = 64
train_iter, test_iter = load_data(HistologyDataset, 'Kather_texture_2016_image_tiles_5000', batch_size, resize=None)

In [ ]:
# IMPLEMENTE AQUI SUA REDE E DEFINIÇÃO DE LOSS E OTIMIZADOR

# experimente criar redes do zero com o conhecimento adquirido no curso até agora
# experimente também replicar redes já estabelecidas (alexnet, lenet, vgg e etc)
# experimente também utilizar as redes pré-treinadas já implementadas no torchvision
# para o caso de rede pré-treinada, lembre-se de modificar a saída da rede para o número de classes do problema

class HistologyCNN(nn.Module):
    def __init__(self, num_classes=8):
        super(HistologyCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 18 * 18, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

num_epochs = 15
lr = 0.001
batch_size = 64
wd_lambda = 0.0001

model = HistologyCNN(num_classes=8).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd_lambda)


# treinamento e validação
train_validate(model, train_iter, test_iter, batch_size, optimizer, loss, num_epochs)

training on cuda
epoch 1, train loss 1.4825, train acc 0.434, test loss 0.8921, test acc 0.666, time 6.5 sec
epoch 2, train loss 0.8576, train acc 0.658, test loss 0.8280, test acc 0.693, time 5.7 sec
epoch 3, train loss 0.6944, train acc 0.730, test loss 0.6709, test acc 0.784, time 5.6 sec
epoch 4, train loss 0.5273, train acc 0.808, test loss 0.6226, test acc 0.794, time 5.6 sec
epoch 5, train loss 0.4405, train acc 0.846, test loss 0.5751, test acc 0.820, time 5.6 sec
epoch 6, train loss 0.3616, train acc 0.864, test loss 0.5897, test acc 0.822, time 5.7 sec
epoch 7, train loss 0.2253, train acc 0.921, test loss 0.7089, test acc 0.766, time 5.7 sec
epoch 8, train loss 0.1475, train acc 0.956, test loss 0.6762, test acc 0.816, time 5.6 sec
epoch 9, train loss 0.1307, train acc 0.957, test loss 0.6948, test acc 0.811, time 5.7 sec
epoch 10, train loss 0.1743, train acc 0.942, test loss 0.7166, test acc 0.825, time 5.6 sec
epoch 11, train loss 0.0699, train acc 0.981, test loss 0.7652

## Problema 2

Neste problema, classificaremos imagens de sensoriamento remoto de plantações de café do dataset público [Brazilian Coffee Scenes](http://www.patreo.dcc.ufmg.br/2017/11/12/brazilian-coffee-scenes-dataset/).
Neste caso, , vamos receber imagens de $64\times 64$ pixels e classificá-las entre duas classes:

1. café, e
2. não café.

In [15]:
# Baixando o dataset
!wget http://www.patreo.dcc.ufmg.br/wp-content/uploads/2017/11/brazilian_coffee_dataset.zip
!unzip -q brazilian_coffee_dataset.zip
;

--2025-07-01 21:16:29--  http://www.patreo.dcc.ufmg.br/wp-content/uploads/2017/11/brazilian_coffee_dataset.zip
Resolving www.patreo.dcc.ufmg.br (www.patreo.dcc.ufmg.br)... 150.164.144.15
Connecting to www.patreo.dcc.ufmg.br (www.patreo.dcc.ufmg.br)|150.164.144.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4796290 (4,6M) [application/zip]
Saving to: ‘brazilian_coffee_dataset.zip’

brazilian_coffee_da 100%[===================>]   4,57M  4,99MB/s    in 0,9s    

2025-07-01 21:16:30 (4,99 MB/s) - ‘brazilian_coffee_dataset.zip’ saved [4796290/4796290]



''

In [ ]:
class CoffeeDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform, train=False, calc_norm=True, has_norm=True):
        self.root = root
        self.train = train
        self.calc_norm = calc_norm
        self.has_norm = has_norm
        self.load_images()
        self.transform = transform

    def load_images(self):
        self.img_list, self.labels = self.read_images(root=self.root)

    def read_images(self, root):
        # para este dataset, existem 5 pastas (fold1, fold2, ..., fold5) com as imagens
        # e existem 5 arquivos txts (fold1.txt, fold2.txt, ..., fold5.txt) com o nome das imagens correspondentes
        # nos arquivos txts, cada linha representa uma imagem seguindo o formato {classe}.{nome da img}
        # sendo classe igual a coffee ou noncoffee (0 ou 1)
        # tratamos o nome das imagens de acordo com cada linha do arquivo (não esquecendo de adicionar o .jpg)
        # convertemos o label em 0 ou 1 dependendo da classe
        # Vamos utilizar o fold 1 como validação e o restante dos folds como treino
        img_list, labels = [], []
        if self.train is True:
          for i in range(1,5):
            data_file = open(os.path.join(root, 'fold' + str(i+1) + '.txt'), "r")  # arquivo com nome das imagens
            data_list = [i.replace('\n', '') for i in data_file.readlines()]
            for row in data_list:
                img_name = '.'.join(row.split('.')[1:])
                img_list.append(os.path.join(root, 'fold' + str(i+1), img_name + '.jpg'))
                labels.append(0 if row.split('.')[0] == 'coffee' else 1)
        else:
            data_file = open(os.path.join(root, 'fold1.txt'), "r")  # arquivo com nome das imagens
            data_list = [i.replace('\n', '') for i in data_file.readlines()]
            for row in data_list:
                img_name = '.'.join(row.split('.')[1:])
                img_list.append(os.path.join(root, 'fold1', img_name + '.jpg'))
                labels.append(0 if row.split('.')[0] == 'coffee' else 1)

        return img_list, labels

    def __getitem__(self, item):
        # retorna uma imagem para o treino/teste
        if self.has_norm is True:
            # normaliza a imagem se has_norm for setado como True
            cur_img = self.normalize_image(self.transform(Image.open(self.img_list[item])))
        else:
            # apenas converte a imagem para tensor, sem normalizar
            cur_img = self.transform(Image.open(self.img_list[item]))
        cur_label = self.labels[item]
        return cur_img, cur_label

    def __len__(self):
        return len(self.img_list)

    def normalize_image(self, img):
        # normaliza uma imagem
        # se calc_norm for True, normaliza pela subtração da média dividida pelo desvio para cada canal da imagem
        # se calc_norm for False, normaliza pelos valores pré-definidos de média e desvio padrão
        if self.calc_norm is True:
            for i in range(img.shape[0]):
                mu = img[i, :, :].mean()
                std = img[i, :, :].std()
                img[i, :, :] = ((img[i, :, :] - mu) / std)
        else:
            img = torchvision.transforms.functional.normalize(img,
                                                mean=torch.Tensor([0.485, 0.456, 0.406]),
                                                std=torch.Tensor([0.229, 0.224, 0.225]))
        return img


def load_data(dataset, root, batch_size, resize=None):
    # o transformer define a sequência de transformações que serão aplicadas na imagem
    # neste caso, a sequência é um redimensionamento da imagem (caso a variável resize seja definida)
    # seguido de uma transformação para tensor
    # várias outras transformações estão disponíveis no Pytorch, como crops, flips, espelhamento e etc.
    transformer = []
    if resize is not None:
        transformer += [torchvision.transforms.Resize(size=(resize,resize))]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    train = dataset(root=root, transform=transformer, train=True) #obtem dataset de treino
    test = dataset(root=root, transform=transformer, train=False) #obtem dataset de validação
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = torch.utils.data.DataLoader(train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers) # criação do dataloader de treino
    test_iter = torch.utils.data.DataLoader(test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers) # criação do dataloader de teste
    return train_iter, test_iter

# carregamento do dado
batch_size = 64
train_iter, test_iter = load_data(CoffeeDataset, 'brazilian_coffee_scenes', batch_size, resize=None)

In [ ]:
# IMPLEMENTE AQUI SUA REDE E DEFINIÇÃO DE LOSS E OTIMIZADOR

# experimente criar redes do zero com o conhecimento adquirido no curso até agora
# experimente também replicar redes já estabelecidas (alexnet, lenet, vgg e etc)
# experimente também utilizar as redes pré-treinadas já implementadas no torchvision
# para o caso de rede pré-treinada, lembre-se de modificar a saída da rede para o número de classes do problema

import torch
import torch.nn as nn
import torch.nn.functional as F

class CoffeeCNN(nn.Module):
    def __init__(self):
        super(CoffeeCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2)

        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 1)

        self.dropout = nn.Dropout(0.6)


    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)), negative_slope=0.1)
        x = self.pool1(x)
        x = self.dropout(x)

        x = F.leaky_relu(self.bn2(self.conv2(x)), negative_slope=0.1)
        x = self.pool2(x)
        x = self.dropout(x)

        x = F.leaky_relu(self.bn3(self.conv3(x)), negative_slope=0.1)
        x = self.pool3(x)
        x = self.dropout(x)

        x = x.view(-1, 128 * 8 * 8)
        x = F.leaky_relu(self.fc1(x), negative_slope=0.1)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))

        return x

num_epochs = 30
lr = 0.001
batch_size = 64


model = CoffeeCNN().to(device)
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

# treinamento e validação
train_validate(model, train_iter, test_iter, batch_size, optimizer, lambda y_hat, y: loss(y_hat, y.float().view(-1, 1)), num_epochs)

training on cuda


epoch 1, train loss 1.5347, train acc 0.500, test loss 0.6231, test acc 0.500, time 1.1 sec
epoch 2, train loss 0.6213, train acc 0.500, test loss 0.4585, test acc 0.500, time 1.1 sec
epoch 3, train loss 0.5214, train acc 0.500, test loss 0.4483, test acc 0.500, time 1.1 sec
epoch 4, train loss 0.4827, train acc 0.500, test loss 0.3604, test acc 0.500, time 1.0 sec
epoch 5, train loss 0.4722, train acc 0.500, test loss 0.3775, test acc 0.500, time 1.0 sec
epoch 6, train loss 0.4274, train acc 0.500, test loss 0.3571, test acc 0.500, time 1.0 sec
epoch 7, train loss 0.4127, train acc 0.500, test loss 0.3626, test acc 0.500, time 1.1 sec
epoch 8, train loss 0.4013, train acc 0.500, test loss 0.3235, test acc 0.500, time 1.1 sec
epoch 9, train loss 0.4120, train acc 0.500, test loss 0.4100, test acc 0.500, time 1.0 sec
epoch 10, train loss 0.4099, train acc 0.500, test loss 0.3396, test acc 0.500, time 1.0 sec
epoch 11, train loss 0.3787, train acc 0.500, test loss 0.3559, test acc 0.500,

## Problema 3

Neste problema, classificaremos imagens gerais de sensoriamento remoto do dataset público [UCMerced](http://weegee.vision.ucmerced.edu/datasets/landuse.html).
Neste caso, vamos receber imagens de $256\times 256$ pixels e classificá-las entre 21 classes:

1. agricultural
1. airplane
1. baseballdiamond
1. beach
1. buildings
1. chaparral
1. denseresidential
1. forest
1. freeway
1. golfcourse
1. harbor
1. intersection
1. mediumresidential
1. mobilehomepark
1. overpass
1. parkinglot
1. river
1. runway
1. sparseresidential
1. storagetanks
1. tenniscourt

In [19]:
# Baixando o dataset
!wget http://weegee.vision.ucmerced.edu/datasets/UCMerced_LandUse.zip
!unzip -q UCMerced_LandUse.zip

--2025-07-01 22:26:26--  http://weegee.vision.ucmerced.edu/datasets/UCMerced_LandUse.zip
Resolving weegee.vision.ucmerced.edu (weegee.vision.ucmerced.edu)... 169.236.184.65
Connecting to weegee.vision.ucmerced.edu (weegee.vision.ucmerced.edu)|169.236.184.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 332468434 (317M) [application/zip]
Saving to: ‘UCMerced_LandUse.zip’

UCMerced_LandUse.zi 100%[===================>] 317,07M  13,2MB/s    in 82s     

2025-07-01 22:27:50 (3,84 MB/s) - ‘UCMerced_LandUse.zip’ saved [332468434/332468434]



In [ ]:
class UCMercedDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform, train=False, calc_norm=False, has_norm=True):
        self.root = root
        self.train = train
        self.calc_norm = calc_norm
        self.has_norm = has_norm
        self.transform = transform
        self.load_images()

    def load_images(self):
        self.img_list, self.labels = self.read_images(root=self.root)

    def read_images(self, root):
        # IMPLEMENTE AQUI A LEITURA DAS IMAGENS

        # para este dataset, as imagens estão dividas em pastas com o nome da classe
        # uma sugestão é usar o enumerate do Python para percorrer essas pastas, atribuindo o valor do enumerate como label da classe
        # outra opção é definir um dicionário com o label de cada classe como no exemplo do Colerectal histology
        # para cada pasta, selecione as primeiras 80 imagens para o treino e o restante para o teste
        img_list, labels = [], []

        if self.train is True:
            for folder in os.listdir(self.root):
                for num, img_name in enumerate(os.listdir(os.path.join(self.root, folder))):
                    if num < 80:
                        img_list.append(os.path.join(self.root, folder, img_name))
                        labels.append(folder)
        else:
            for folder in os.listdir(self.root):
                for num, img_name in enumerate(os.listdir(os.path.join(self.root, folder))):
                    if num >= 80:
                        img_list.append(os.path.join(self.root, folder, img_name))
                        labels.append(folder)

        return img_list, labels

    def __getitem__(self, item):
        # IMPLEMENTE AQUI O RETORNO E TRATAMENTO DE CADA IMAGEM

        # lembre-se de aplicar as transformações enviadas ao dataloader (principalmente o ToTensor)
        

        return img, label

    def __len__(self):
        return len(self.img_list)

    def normalize_image(self, img):
        # normaliza uma imagem
        # se calc_norm for True, normaliza pela subtração da média dividida pelo desvio para cada canal da imagem
        # se calc_norm for False, normaliza pelos valores pré-definidos de média e desvio padrão
        if self.calc_norm is True:
            for i in range(img.shape[0]):
                mu = img[i, :, :].mean()
                std = img[i, :, :].std()
                img[i, :, :] = ((img[i, :, :] - mu) / std)
        else:
            img = torchvision.transforms.functional.normalize(img,
                                                mean=torch.Tensor([0.485, 0.456, 0.406]),
                                                std=torch.Tensor([0.229, 0.224, 0.225]))
        return img


def load_data(dataset, root, batch_size, resize=None):
    # IMPLEMENTE AQUI A DEFINIÇÃO DAS TRANSFORMAÇÕES E DO DATALOADER

    # o transformer define a sequência de transformações que serão aplicadas na imagem
    # a principal para o nosso caso é o ToTensor, que converte a imagem no formato lido para um tensor
    # experimente transformações diferentes, como crops e flips
    # o resize pode ser necessário para datasets com imagems de tamanhos variados

    # defina também o dataloader de treino e teste

    return train_iter, test_iter

# carregamento do dado
batch_size = 64
train_iter, test_iter = load_data(UCMercedDataset, os.path.join('UCMerced_LandUse', 'Images'), batch_size, resize=256)

In [ ]:
# IMPLEMENTE AQUI SUA REDE E DEFINIÇÃO DE LOSS E OTIMIZADOR

# experimente criar redes do zero com o conhecimento adquirido no curso até agora
# experimente também replicar redes já estabelecidas (alexnet, lenet, vgg e etc)
# experimente também utilizar as redes pré-treinadas já implementadas no torchvision
# para o caso de rede pré-treinada, lembre-se de modificar a saída da rede para o número de classes do problema

# treinamento e validação
train_validate(model, train_iter, test_iter, batch_size, optimizer, loss, num_epochs)

## Problema 4

Neste problema, classificaremos imagens genéricas de textura do dataset público [*Describable Textures Dataset*](http://www.robots.ox.ac.uk/~vgg/data/dtd/).
Neste caso, vamos receber imagens com tamanho variado (de $300\times 300$ pixels até $640\times 640$) e classificá-las entre 47 classes:

1.  banded
1.  blotchy
1.  braided
1.  bubbly
1.  bumpy
1.  chequered
1.  cobwebbed
1.  cracked
1.  crosshatched
1.  crystalline
1.  dotted
1.  fibrous
1.  flecked
1.  freckled
1.  frilly
1.  gauzy
1.  grid
1.  grooved
1.  honeycombed
1.  interlaced
1.  knitted
1.  lacelike
1.  lined
1.  marbled
1.  matted
1.  meshed
1.  paisley
1.  perforated
1.  pitted
1.  pleated
1.  polka-dotted
1.  porous
1.  potholed
1.  scaly
1.  smeared
1.  spiralled
1.  sprinkled
1.  stained
1.  stratified
1.  striped
1.  studded
1.  swirly
1.  veined
1.  waffled
1.  woven
1.  wrinkled
1.  zigzagged

In [ ]:
# Download do dataset
!wget http://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz
!tar -xzf dtd-r1.0.1.tar.gz

In [ ]:
class TextureDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform, train=False, calc_norm=False, has_norm=True):
        self.root = root
        self.train = train
        self.calc_norm = calc_norm
        self.has_norm = has_norm
        self.le = {'banded': 0, 'blotchy': 1, 'braided': 2, 'bubbly': 3, 'bumpy': 4,
                   'chequered': 5, 'cobwebbed': 6, 'cracked': 7, 'crosshatched': 8,
                   'crystalline': 9, 'dotted': 10, 'fibrous': 11, 'flecked': 12,
                   'freckled': 13, 'frilly': 14, 'gauzy': 15, 'grid': 16, 'grooved': 17,
                   'honeycombed': 18, 'interlaced': 19, 'knitted': 20, 'lacelike': 21, 'lined': 22,
                   'marbled': 23, 'matted': 24, 'meshed': 25, 'paisley': 26, 'perforated': 27,
                   'pitted': 28, 'pleated': 29, 'polka-dotted': 30, 'porous': 31, 'potholed': 32,
                   'scaly': 33, 'smeared': 34, 'spiralled': 35, 'sprinkled': 36, 'stained': 37,
                   'stratified': 38, 'striped': 39, 'studded': 40, 'swirly': 41, 'veined': 42,
                   'waffled': 43, 'woven': 44, 'wrinkled': 45, 'zigzagged': 46} # dicionário definindo o label de cada classe
        self.transform = transform
        self.load_images()

    def load_images(self):
        self.img_list, self.labels = self.read_images(root=self.root)

    def read_images(self, root):
        # IMPLEMENTE AQUI A LEITURA DAS IMAGENS

        # para este caso, na pasta images estão as imagens separadas por pastas relacionadas a classe
        # na pasta label existem txts definindo a divisão das imagens em treino, teste e validação
        # utilize as imagens nos arquivos train1.txt e val1.txt como treino
        # utilize as imagens nos arquivos teste1.txt como validação
        # lembre-se de atribuir o label de acordo com o dicionário self.le definido acima

        img_list, labels = [], []

        #...

        return img_list, labels

    def __getitem__(self, item):
        # IMPLEMENTE AQUI O RETORNO E TRATAMENTO DE CADA IMAGEM

        # lembre-se de aplicar as transformações enviadas ao dataloader (principalmente o ToTensor)

        return img, label

    def __len__(self):
        return len(self.img_list)

    def normalize_image(self, img):
        # normaliza uma imagem
        # se calc_norm for True, normaliza pela subtração da média dividida pelo desvio para cada canal da imagem
        # se calc_norm for False, normaliza pelos valores pré-definidos de média e desvio padrão
        if self.calc_norm is True:
            for i in range(img.shape[0]):
                mu = img[i, :, :].mean()
                std = img[i, :, :].std()
                img[i, :, :] = ((img[i, :, :] - mu) / std)
        else:
            img = torchvision.transforms.functional.normalize(img,
                                                mean=torch.Tensor([0.485, 0.456, 0.406]),
                                                std=torch.Tensor([0.229, 0.224, 0.225]))
        return img


def load_data(dataset, root, batch_size, resize=None):
    # IMPLEMENTE AQUI A DEFINIÇÃO DAS TRANSFORMAÇÕES E DO DATALOADER

    # o transformer define a sequência de transformações que serão aplicadas na imagem
    # a principal para o nosso caso é o ToTensor, que converte a imagem no formato lido para um tensor
    # experimente transformações diferentes, como crops e flips
    # o resize pode ser necessário para datasets com imagems de tamanhos variados

    # defina também o dataloader de treino e teste

    return train_iter, test_iter

# carregamento do dado
batch_size = 32
train_iter, test_iter = load_data(TextureDataset, os.path.join('dtd'), batch_size, resize=None)

In [ ]:
# IMPLEMENTE AQUI SUA REDE E DEFINIÇÃO DE LOSS E OTIMIZADOR

# experimente criar redes do zero com o conhecimento adquirido no curso até agora
# experimente também replicar redes já estabelecidas (alexnet, lenet, vgg e etc)
# experimente também utilizar as redes pré-treinadas já implementadas no torchvision
# para o caso de rede pré-treinada, lembre-se de modificar a saída da rede para o número de classes do problema

# treinamento e validação
train_validate(model, train_iter, test_iter, batch_size, optimizer, loss, num_epochs)

In [ ]:
# função auxiliar para plotar imagens do dataset

from matplotlib import pyplot as plt

def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):
    """Plot a list of images."""
    figsize = (num_cols * scale, num_rows * scale)
    axes = plt.subplots(num_rows, num_cols, figsize=figsize)[1].flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        ax.imshow(img.numpy())
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

In [ ]:
#plota imagens do dataset

imgs = []
for X, y in train_iter:
    X = np.swapaxes(X, 1, 3)
    imgs = X[0:18]
    break


show_images(imgs, 3, 6, titles=None, scale=3)